## Preliminaries

In [172]:
# Dependencies
import pickle
import json
from openai import OpenAI
import numpy as np
import os
import pandas as pd
import time

In [173]:
# Assume you have a file named 'example.pkl' containing a pickled object
with open('ssdh_all_raw_message', 'rb') as file:
    # Load the pickled object back into Python
    unpickled_object = pickle.load(file)

# The client for the API
client = OpenAI()

In [174]:
# Document slicing
documents = []
for i in range(0, int(len(unpickled_object)/2)):
    documents.append(unpickled_object[2*i])

## Single prompt test for the API

In [111]:
# Original, non-modified prompt
prompt_file = open('prompts/original.txt','r').read()

# Assistant creation
assistant = client.beta.assistants.create(
    name="Chilean History Expert",
    instructions=prompt_file,
    model="gpt-4" # gpt-4 turbo is x2 times exprensive
)

In [158]:
# 'Conversation thread' creation
thread = client.beta.threads.create()

# Single message added
msg = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=documents[19]
)

# Run the model
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [162]:
# Check if the run is completed
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print(run.status)

completed


In [161]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
messages.data[0].content[0].text.value

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_2xs5MBcLdvOVrOdEoG4Wo5Pl', assistant_id='asst_OnVlScN4AOSqVpRvDSA0gLZ4', content=[MessageContentText(text=Text(annotations=[], value="{'Gregorio BRAVO Ponce de Leon': [('Position within prison', 'Member of the Well-being Committee responsible for improving detention conditions'), ('Location', 'Held in the prison characterized by overcrowding, insufficient hygienic conditions and insufficient medical attention, particularly in case of emergencies')],\n 'Carlos RIOS Fuentes': [('Position within prison', 'Member of the Well-being Committee responsible for improving detention conditions'), ('Location', 'Held in the prison characterized by overcrowding, insufficient hygienic conditions and insufficient medical attention, particularly in case of emergencies')],\n 'German CALLEJAS Guerrero': [('Position within prison', 'Member of the Well-being Committee responsible for improving detention conditions'), ('Location', 'Held in the prison

## Prompt engineering tactics
The idea is to follow some of the [strategies given by OpenAI](https://platform.openai.com/docs/guides/prompt-engineering) for getting better results using prompt engineering. 
The original prompt will be used as a base.

### Write clear instructions
1. Let the model guess as little as possible.
2. Step-by-step instructions (also trying the 'magical' sentence [Take a deep breath and work on this problem step-by-step](https://arxiv.org/pdf/2309.03409.pdf)).
### Split complex tasks into simpler subtasks
3. Recursively summarize and clean text.
### Give the model time to "think"
4. Ask if there are any relevant info that was left behind. Tell them to not repeat info.
(ex. Are there more relevant excerpts? Take care not to repeat excerpts. Also ensure that excerpts contain all relevant context needed to interpret them - in other words don't extract small snippets that are missing important context.)

### TBA
#### Use external tools
- APIs.
- Functions.
#### Test changes systematically
- [https://github.com/openai/evals/tree/main]
- [https://www.deeplearning.ai/short-courses/automated-testing-llmops/]

## Prompts description
Row 0 correspond to the different prompts written in the './prompts' directory.

Column 0 corresond to the strategies presented above, each represented with their respective number.
|    | var1 | var2 | var3 | var4 | var5 | var6 | var7 | var8 |
|----|------|------|------|------|------|------|------|------|
| (1)|  ✅  |  ✅   | ✅   | ✅   | ✅   | ✅   | ✅   | ✅  | 
| (2)|  -   |  ✅  |  -   |  ✅   | -   | - | ✅   | ✅ |  
| (3)| -   | -   | ✅   |  ✅  | - | ✅   | -   | ✅   |
| (4)| -   | -   | -   |  -  | ✅   | ✅   | ✅   | ✅   |

In [175]:
# 3, 5, 11, 17, 19 will be used as sample for manual evaluation
# document 17 gets no result from the original prompt (added on purpose)
documents_sample = []
for i in [3, 5, 11, 17, 19]:
    documents_sample.append(documents[i])

directory = "./prompts/"
prompts = []
for prompt in os.listdir(directory):
    with open(os.path.join(directory, prompt), 'r', encoding='utf-8') as file:
        prompts.append(file.read())

In [184]:
def results_to_df(prompts_batch, data_samples, openai_client):

    df = pd.DataFrame(columns=['prompt', 'document', 'result']) # df to return

    for prompt_idx, prompt in enumerate(prompts_batch):

        assistant = openai_client.beta.assistants.create(
            name="Chilean History Expert",
            instructions=prompt,
            model="gpt-4"
        )

        for doc_idx, doc in enumerate(data_samples):
            thread = openai_client.beta.threads.create()
            print(f"new thread created with id: {thread.id}.")

            m = openai_client.beta.threads.messages.create(
                thread_id=thread.id,
                role="user",
                content=doc
            )
            print(f"new message created with id: {m.id}.")

            run = openai_client.beta.threads.runs.create(
                thread_id=thread.id,
                assistant_id=assistant.id,
            )

            print(f"run instance created with id: {run.id}.")

            # wait while the assistant completes the answer
            run_status = ''
            while run_status != 'completed':
                print(f"run status: {run_status}.")
                run_status = openai_client.beta.threads.runs.retrieve(
                    thread_id=thread.id,
                    run_id=run.id
                ).status
                time.sleep(3)
            print(f"run status: {run_status}.")

            # assistant answer
            msg = openai_client.beta.threads.messages.list(
                thread_id=thread.id
            ).data[0].content[0].text.value

            # append row to df
            row = {'prompt' : prompt_idx, 'document' : doc_idx, 'result' : msg}
            df = pd.concat([df, pd.DataFrame.from_records([row])])
            print(f"Document {doc_idx} done.")

        print(f"Prompt {prompt_idx} done.")

    return df


In [185]:
df_results = results_to_df(prompts, documents_sample, client)
df_results

new thread created with id: thread_NAasKK5WtoaAd2JOkIZIACHk.
new message created with id: msg_84UtpDl5NL4zy7sUkOo86JVI.
run instance created with id: run_G9WG1a383PxDS6glNAUxmzX3.
run status: .
run status: in_progress.
run status: in_progress.
run status: in_progress.
run status: in_progress.
run status: in_progress.
run status: in_progress.
Document 0 done.
new thread created with id: thread_rkSs7dVhSsWrkDAm0AIp8JgB.
new message created with id: msg_hAsAaPBF6v0NzTlGmMdDwcLj.
run instance created with id: run_AcnWvZP9jLUjOIxlk5jbJKDV.
run status: .
run status: in_progress.
run status: in_progress.
run status: in_progress.
run status: in_progress.
run status: in_progress.
run status: in_progress.
run status: in_progress.
run status: in_progress.
run status: in_progress.
run status: in_progress.
run status: in_progress.
run status: in_progress.
run status: in_progress.
run status: in_progress.
run status: in_progress.
Document 1 done.
new thread created with id: thread_NO5sgLCdzHlfzhMH5J

,prompt,document,result
0,0,0,"{'Eddi Leemann': [('Role', 'Delegate of the In..."
0,0,1,"{""Manuel Contreras Sepulveda"": [(""Role"", ""Dire..."
0,0,2,Step 1 - Document Summary:\nThe document appea...
0,0,3,Given the lack of specific people involved in ...
0,0,4,{'Gregorio Bravo Ponce de Leon': [('Role in pr...
0,1,0,{'Eddi Leemann': [('Visited Puchuncaví detenti...
0,1,1,{'Julio Arturo Acosta Georges': [('Arrested by...
0,1,2,"{'Jorge GONZALEZ Morales': [('Condition', 'Oti..."
0,1,3,The provided document does not contain specifi...
0,1,4,"{'Gregorio BRAVO Ponce de Leon': [('Role', 'Me..."


In [186]:
df_results.to_csv('results.csv')